In [1]:
include("../LiPoSID.jl")
using QuantumOptics
basis = NLevelBasis(2)
using LinearAlgebra
using Plots
using StatsPlots
using Dates
using HDF5

In [12]:
function propagate(A, x0, steps)
    
    x = []
    push!(x, x0)

    @assert size(x0,1) == size(A,1) == size(A,2)

    for i=2:steps
        push!(x, A * x[end])
    end

    return x
end 

function bloch(ρ_list)
    # Pauli matricies
    σ = [ [0 1; 1 0], [0 -im; im 0], [1 0; 0 -1], [1 0; 0 1] ]

    bloch_vec = [
        convert.(Float64, [ ρ[1,2]+ρ[2,1],
                           (ρ[1,2]-ρ[2,1])*im,
                            ρ[1,1]-ρ[2,2]       ])
                for ρ in ρ_list]
    hcat(bloch_vec...)
end

function propagate_rho_O1XY(ρ₀, ρᵉᵍˣʸ, steps)

    #hcat(vec[ρᵢ] for ρᵢ in ρᵉᵍˣʸ)

    kᵉᵍˣʸ = hcat([vec(ρᵢ[1]) for ρᵢ in ρᵉᵍˣʸ]...)\vec(ρ₀)

    kᵉ, kᵍ, kˣ, kʸ = kᵉᵍˣʸ              
    ρᵉ, ρᵍ, ρˣ, ρʸ = ρᵉᵍˣʸ

    steps = min(length(ρᵉ), length(ρᵍ), length(ρˣ), length(ρʸ))

    ρ = kᵉ * ρᵉ[1:steps] + kᵍ * ρᵍ[1:steps] + kˣ * ρˣ[1:steps] + kʸ * ρʸ[1:steps]

    return ρ
end 

propagate_rho_O1XY (generic function with 1 method)

In [17]:
function LMEvs4DBlochDMD_basis(γᵢ)

    data_dir = "../DATA/"

    γᵢ = string(γᵢ)

    if γᵢ == "0.0" 
        ρᵉ, tᵉ = LiPoSID.get_rho_series(data_dir*"State_B2_2CUT_data.h5", "0.079477")
        ρᵍ, tᵍ = LiPoSID.get_rho_series(data_dir*"State_B1_2CUT_data.h5", "0.079477")
        ρˣ, tˣ = LiPoSID.get_rho_series(data_dir*"State_B3_2CUT_data.h5", "0.079477")
        ρʸ, tʸ = LiPoSID.get_rho_series(data_dir*"State_B4_2CUT_data.h5", "0.079477")
    else 
        ρᵉ, tᵉ = LiPoSID.get_rho_series(data_dir*"State_B2_2CUT_data.h5", γᵢ)
        ρᵍ, tᵍ = LiPoSID.get_rho_series(data_dir*"State_B1_2CUT_data.h5", γᵢ)
        ρˣ, tˣ = LiPoSID.get_rho_series(data_dir*"State_B3_2CUT_data.h5", γᵢ)
        ρʸ, tʸ = LiPoSID.get_rho_series(data_dir*"State_B4_2CUT_data.h5", γᵢ)
    end

    lᵉ = length(ρᵉ); lᵍ = length(ρᵍ); lˣ = length(ρˣ); lʸ = length(ρʸ)
    lᵐᵃˣ = min(lᵉ, lᵍ,  lˣ, lʸ)  #choose time limit by shortest series

    bᵉ = bloch(ρᵉ[1:lᵐᵃˣ])
    bᵍ = bloch(ρᵍ[1:lᵐᵃˣ])
    bˣ = bloch(ρˣ[1:lᵐᵃˣ])
    bʸ = bloch(ρʸ[1:lᵐᵃˣ])

    ω = 25.126

    Hᴸᴹᴱ = [ ω        0
             0        0   ]
    
    γ = parse(Float64, γᵢ)

    Aᴸᴹᴱ = [ 0    √γ
             0     0. + 0im  ]

    tᵉᶠ = convert.(Float64, tᵉ); tᵍᶠ = convert.(Float64, tᵍ); 
    tˣᶠ = convert.(Float64, tˣ); tʸᶠ = convert.(Float64, tʸ); 

    t_list = [tᵉᶠ, tᵍᶠ, tˣᶠ, tʸᶠ]

    t = t_list[argmin(length(tⁱ) for tⁱ in t_list)]

    ρᵉᴸᴹᴱ = LiPoSID.Lindblad_time_evolution(basis, ρᵉ[1], tᵉᶠ, Hᴸᴹᴱ, [Aᴸᴹᴱ])
    ρᵍᴸᴹᴱ = LiPoSID.Lindblad_time_evolution(basis, ρᵍ[1], tᵍᶠ, Hᴸᴹᴱ, [Aᴸᴹᴱ])
    ρˣᴸᴹᴱ = LiPoSID.Lindblad_time_evolution(basis, ρˣ[1], tˣᶠ, Hᴸᴹᴱ, [Aᴸᴹᴱ])
    ρʸᴸᴹᴱ = LiPoSID.Lindblad_time_evolution(basis, ρʸ[1], tʸᶠ, Hᴸᴹᴱ, [Aᴸᴹᴱ])

    ρᴸᴹᴱ = [ρᵉᴸᴹᴱ, ρᵍᴸᴹᴱ, ρˣᴸᴹᴱ, ρʸᴸᴹᴱ]

    bᵉᴸᴹᴱ = LiPoSID.bloch(ρᵉᴸᴹᴱ[1:lᵐᵃˣ])
    bᵍᴸᴹᴱ = LiPoSID.bloch(ρᵍᴸᴹᴱ[1:lᵐᵃˣ])
    bˣᴸᴹᴱ = LiPoSID.bloch(ρˣᴸᴹᴱ[1:lᵐᵃˣ])
    bʸᴸᴹᴱ = LiPoSID.bloch(ρʸᴸᴹᴱ[1:lᵐᵃˣ])

    Yᵉ = [bᵉᴸᴹᴱ; ones(lᵐᵃˣ)']
    Yᵍ = [bᵍᴸᴹᴱ; ones(lᵐᵃˣ)']
    Yˣ = [bˣᴸᴹᴱ; ones(lᵐᵃˣ)']
    Yʸ = [bʸᴸᴹᴱ; ones(lᵐᵃˣ)']

    Yᵉ⁻ = Yᵉ[:,1:end-1]; Yᵉ⁺ = Yᵉ[:,2:end]
    Yᵍ⁻ = Yᵍ[:,1:end-1]; Yᵍ⁺ = Yᵍ[:,2:end]
    Yˣ⁻ = Yˣ[:,1:end-1]; Yˣ⁺ = Yˣ[:,2:end]
    Yʸ⁻ = Yᵉ[:,1:end-1]; Yʸ⁺ = Yᵉ[:,2:end]

    Y⁻ = hcat(Yᵉ⁻, Yᵍ⁻, Yˣ⁻, Yʸ⁻)
    Y⁺ = hcat(Yᵉ⁺, Yᵍ⁺, Yˣ⁺, Yʸ⁺)

    A⁴ᴰ = Y⁺ * pinv(Y⁻) # Direct DMD estimation
    
    bᵉ⁴ᴰ = hcat(propagate(A⁴ᴰ, [bᵉ[:,1]...,1], lᵐᵃˣ)...)[1:3,:] 
    bᵍ⁴ᴰ = hcat(propagate(A⁴ᴰ, [bᵍ[:,1]...,1], lᵐᵃˣ)...)[1:3,:]
    bˣ⁴ᴰ = hcat(propagate(A⁴ᴰ, [bˣ[:,1]...,1], lᵐᵃˣ)...)[1:3,:]
    bʸ⁴ᴰ = hcat(propagate(A⁴ᴰ, [bʸ[:,1]...,1], lᵐᵃˣ)...)[1:3,:]

    ρᵉ⁴ᴰ = LiPoSID.rho_series_from_bloch(bᵉ⁴ᴰ) 
    ρᵍ⁴ᴰ = LiPoSID.rho_series_from_bloch(bᵍ⁴ᴰ)
    ρˣ⁴ᴰ = LiPoSID.rho_series_from_bloch(bˣ⁴ᴰ)
    ρʸ⁴ᴰ = LiPoSID.rho_series_from_bloch(bʸ⁴ᴰ)

    ρᴮ⁴ᴰᴹᴰ = [ρᵉ⁴ᴰ, ρᵍ⁴ᴰ, ρˣ⁴ᴰ, ρʸ⁴ᴰ]

    #Ac⁴ᴰ = [  - γ/2   -ω       0   0   # This is better 0.9999999993089329
    #            ω     -γ/2     0   0
    #            0.     0      -γ   γ
    #            0      0       0   0]

    Ac⁴ᴰ⁻ᴬᴺᴬᴸᴵᵀ = [  0     -ω     0   0      # This is also good 0.9999890166421603 ⁻
                     ω     -γ     0   0
                     0.     0    -γ   γ
                     0      0     0   0 ]

    Δt = t[2] - t[1]

    A⁴ᴰᴬ = exp(Ac⁴ᴰ⁻ᴬᴺᴬᴸᴵᵀ  * Δt) 
      
    bᵉ⁴ᴰᴬ = hcat(propagate(A⁴ᴰᴬ, [bᵉ[:,1]...,1], length(t))...)[1:3,:] 
    bᵍ⁴ᴰᴬ = hcat(propagate(A⁴ᴰᴬ, [bᵍ[:,1]...,1], length(t))...)[1:3,:]
    bˣ⁴ᴰᴬ = hcat(propagate(A⁴ᴰᴬ, [bˣ[:,1]...,1], length(t))...)[1:3,:]
    bʸ⁴ᴰᴬ = hcat(propagate(A⁴ᴰᴬ, [bʸ[:,1]...,1], length(t))...)[1:3,:]

    ρᵉ⁴ᴰᴬ = LiPoSID.rho_series_from_bloch(bᵉ⁴ᴰᴬ) 
    ρᵍ⁴ᴰᴬ = LiPoSID.rho_series_from_bloch(bᵍ⁴ᴰᴬ)
    ρˣ⁴ᴰᴬ = LiPoSID.rho_series_from_bloch(bˣ⁴ᴰᴬ)
    ρʸ⁴ᴰᴬ = LiPoSID.rho_series_from_bloch(bʸ⁴ᴰᴬ)

    ρᴮ⁴ᴬ = [ρᵉ⁴ᴰ, ρᵍ⁴ᴰ, ρˣ⁴ᴰ, ρʸ⁴ᴰ]
    
    return ρᴸᴹᴱ, ρᴮ⁴ᴰᴹᴰ, ρᴮ⁴ᴬ, t[1:lᵐᵃˣ]

end

LMEvs4DBlochDMD_basis (generic function with 1 method)

In [18]:
parentdir = ".."
data_dir = parentdir*"/DATA/"
println(data_dir)

models_dir = parentdir*"/MODELS/"
tests_dir = parentdir*"/TESTS/"

dodeca_files = ["State_D"*string(n) for n=1:20];
basis_files = ["State_B"*string(n) for n=1:4];

all_files = vcat(dodeca_files, basis_files)
test_files = dodeca_files;

../DATA/


In [19]:
date_and_time_string =  string(Dates.format(now(), "yyyy-u-dd_at_HH-MM"))

tests_data_file_name = "DMD-Bloch4D_LME_trn4_tst20_"*date_and_time_string * ".h5"

#γ = [ "0.079477",  "0.25133", "0.79477", "2.5133", "7.9477", "25.133",  "79.477", "251.33"]

γ = ["0.0", "0.079477",  "0.25133", "0.79477", "2.5133", "7.9477", "25.133",  "79.477", "251.33"]

for γᵢ in  γ

    h5open(tests_dir*tests_data_file_name,"cw") do fid
        γ_group = create_group(fid, "gamma_"*γᵢ) # create coupling group
    end 

    ρᴸᴹᴱ, ρᴮ⁴ᴰᴹᴰ, ρᴮ⁴ᴬ, t = LMEvs4DBlochDMD_basis(γᵢ)

    println("γᵢ = ", γᵢ)

    ρᵉᴸᴹᴱ, ρᵍᴸᴹᴱ, ρˣᴸᴹᴱ, ρʸᴸᴹᴱ = ρᴸᴹᴱ
    ρᵉᴮ⁴ᴰᴹᴰ, ρᵍᴮ⁴ᴰᴹᴰ, ρˣᴮ⁴ᴰᴹᴰ, ρʸᴮ⁴ᴰᴹᴰ = ρᴮ⁴ᴰᴹᴰ
    ρᵉᴮ⁴ᴬ, ρᵍᴮ⁴ᴬ, ρˣᴮ⁴ᴬ, ρʸᴮ⁴ᴬ = ρᴮ⁴ᴬ

    Hᴸᴹᴱ = [ 25.126        0
             0             0    ]

    γᶠ = parse(Float64, γᵢ)

    Aᴸᴹᴱ = [ 0    √(γᶠ)
             0     0. + 0im  ]

    for df_tst in test_files # loop over initial states
        
        print(df_tst*" ")

        if parse(Float64, γᵢ) < 0.079 # γᵢ == "0.0" 
            ρᵗˢᵗ, tᵗˢᵗ = LiPoSID.get_rho_series(data_dir*df_tst*"_2CUT_data.h5", "0.079477")

        else 
            ρᵗˢᵗ, tᵗˢᵗ = LiPoSID.get_rho_series(data_dir*df_tst*"_2CUT_data.h5", γᵢ)
        end

        if length(tᵗˢᵗ) > 1200 end_tst = 1200 else end_tst = length(tᵗˢᵗ) end

        tᵗˢᵗᶠ = convert.(Float64, tᵗˢᵗ)

        ρᵗˢᵗᴸᴹᴱ = LiPoSID.Lindblad_time_evolution(basis, ρᵗˢᵗ[1], tᵗˢᵗᶠ, Hᴸᴹᴱ, [Aᴸᴹᴱ])
        ρᵗˢᵗᴸᴹᴱ = convert(Vector{Matrix{ComplexF64}}, ρᵗˢᵗᴸᴹᴱ[1:end_tst])

        steps = min(end_tst, length(t))

        ρᵗˢᵗᴮ⁴ᴰᴹᴰ =  propagate_rho_O1XY(ρᵗˢᵗ[1], ρᴮ⁴ᴰᴹᴰ, steps)
        ρᵗˢᵗᴮ⁴ᴬ =  propagate_rho_O1XY(ρᵗˢᵗ[1], ρᴮ⁴ᴬ, steps)

        Fᴮ⁴ᴰᴹᴰ = LiPoSID.fidelity_series(basis, ρᵗˢᵗᴮ⁴ᴰᴹᴰ[1:steps], ρᵗˢᵗᴸᴹᴱ[1:steps])
        Fᴮ⁴ᴬ = LiPoSID.fidelity_series(basis, ρᵗˢᵗᴮ⁴ᴬ[1:steps], ρᵗˢᵗᴸᴹᴱ[1:steps])

        h5open(tests_dir*tests_data_file_name,"cw") do fid
            γ_group = open_group(fid, "gamma_"*γᵢ) # create coupling group
            state_group = create_group(γ_group, df_tst) # create coupling group
            state_group["F_b4dmd_lme"] = convert.(Float64, Fᴮ⁴ᴰᴹᴰ)
            state_group["F_b4a_lme"] = convert.(Float64, Fᴮ⁴ᴬ)
            state_group["time"] = convert.(Float64, tᵗˢᵗ[1:steps]) 
            
        end
    end
end

γᵢ = 0.0
State_D1 State_D2 State_D3 

State_D4 State_D5 State_D6 State_D7 

State_D8 State_D9 State_D10 State_D11 

State_D12 State_D13 State_D14 State_D15 

State_D16 State_D17 State_D18 State_D19 State_D20 γᵢ = 

...

79.477
State_D1 State_D2 State_D3 State_D4 State_D5 State_D6 State_D7 State_D8 

State_D9 State_D10 State_D11 State_D12 State_D13 State_D14 State_D15 State_D16 

State_D17 State_D18 State_D19 State_D20 

γᵢ = 251.33
State_D1 State_D2 State_D3 State_D4 State_D5 State_D6 State_D7 State_D8 State_D9 State_D10 State_D11 State_D12 State_D13 State_D14 State_D15 

State_D16 State_D17 State_D18 State_D19 State_D20 